In [3]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("all_co_id_info.csv", quotechar='"', sep=",", encoding="utf-8")

In [5]:
from datetime import datetime

def convert_to_date(date_str):
    try:
        if date_str and isinstance(date_str, str):  # Kiểm tra nếu date_str không rỗng và là chuỗi
            return datetime.strptime(date_str.strip(), '%d/%m/%Y').date()  # Định dạng dd/mm/yyyy
        return None  # Trả về None nếu không có ngày
    except ValueError:
        return None  # Trả về None nếu không thể chuyển đổi

    
def split_address(address_list):
    name = address_list[0].strip() if address_list else ''
    address1 = address_list[1].strip() if len(address_list) > 1 else ''
    address2 = address_list[2].strip() if len(address_list) > 2 else ''
    return name, address1, address2

In [6]:
df['Issuance date'] = df['Issuance date'].apply(convert_to_date)
df['Certified date'] = df['Certified date'].apply(convert_to_date)
df['Departure Date'] = df['Departure Date'].apply(convert_to_date)

In [7]:
import ast

columns_to_convert = ['ExportDeclaration', 'goods_info', 'origindoc','invoice_attached','remarks_info','Goods consigned from','Goods consigned to']
for col in columns_to_convert:
    if col in df.columns:  # Kiểm tra nếu cột tồn tại
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

In [8]:
df = df.fillna('')

In [31]:
import pyodbc

# Kết nối tới SQL Server
server = 'localhost,1434'
username = 'sa'
password = '1234QWER@'


In [34]:

# Kết nối đến database master để tạo database mới
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=master;UID={username};PWD={password}')
cursor = connection.cursor()

# Tạo database nếu chưa tồn tại
try:
    cursor.execute("IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'COData') "
                   "BEGIN "
                   "CREATE DATABASE COData; "
                   "END")
    connection.commit()  # Commit lệnh CREATE DATABASE
except Exception as e:
    print("Lỗi khi tạo cơ sở dữ liệu:", e)

# Đóng kết nối và mở lại kết nối tới database mới
connection.close()

# Kết nối đến database mới
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=COData;UID={username};PWD={password}')
cursor = connection.cursor()

# Tạo bảng C/O
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'CO')
    BEGIN
        CREATE TABLE CO (
            doc_id NVARCHAR(50) PRIMARY KEY,
            Trạng_thái_hồ_sơ NVARCHAR(MAX),
            Tax_code NVARCHAR(50),
            Issuing_Authority NVARCHAR(100),
            Form NVARCHAR(50),
            Importing_Country NVARCHAR(100),
            Reference_No NVARCHAR(100),
            Issuance_date DATE,
            Certified_date DATE,
            total_value NVARCHAR(50),
            total_quantity NVARCHAR(50),
            total_weight NVARCHAR(50),
            declaration_place NVARCHAR(100),
            is_show_fob_value bit
        );
    END
""")
#tạo bảng transport
cursor.execute("""

    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Transport')
    BEGIN
        CREATE TABLE Transport (
            doc_id NVARCHAR(50),
            transport_type NVARCHAR(50),
            departure_date DATE,
            vessel_aircraft_name NVARCHAR(100),
            port_of_loading NVARCHAR(100),
            port_of_discharge NVARCHAR(100),
            transport_doc NVARCHAR(MAX),
            CONSTRAINT FK_Transport_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END               
               
""")
#tạo bảng from_to
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'from_to')
    BEGIN
        CREATE TABLE from_to (
            doc_id NVARCHAR(50),
            name_export NVARCHAR(100),
            address_export NVARCHAR(200),
            address_export2 NVARCHAR(200),
            name_import NVARCHAR(100),
            address_import NVARCHAR(200),
            address_import2 NVARCHAR(200),
            CONSTRAINT FK_from_to_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")

# Tạo bảng ExportDeclaration
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'ExportDeclaration')
    BEGIN
        CREATE TABLE ExportDeclaration (
            doc_id NVARCHAR(50),
            item_num INT,
            invoice_number NVARCHAR(50),
            invoice_date DATE,
            invoice_link NVARCHAR(MAX),
            CONSTRAINT FK_ExportDeclaration_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")

# Tạo bảng Goods_info
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Goods_info')
    BEGIN
        CREATE TABLE Goods_info (
            doc_id NVARCHAR(50),
            item_num INT,
            item_number INT,
            item_id NVARCHAR(50),
            Marks_and_numbers_on_packages NVARCHAR(100),
            Goods_description NVARCHAR(200),
            Origin_criterion NVARCHAR(50),
            FOB_value NVARCHAR(100),
            Invoice_number_and_date NVARCHAR(100),
            CONSTRAINT FK_Goods_info_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")

# Tạo bảng Remarks_info
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Remarks_info')
    BEGIN
        CREATE TABLE Remarks_info (
            doc_id NVARCHAR(50),
            label NVARCHAR(100),
            checked BIT,
            CONSTRAINT FK_Remarks_info_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")


# Tạo bảng origindoc
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'origindoc')
    BEGIN
        CREATE TABLE origindoc (
            doc_id NVARCHAR(50),
            origindoc VARCHAR(max),
            
            CONSTRAINT FK_origindoc_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")

# Tạo bảng invoice_attached
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'invoice_attached')
    BEGIN
        CREATE TABLE invoice_attached (
            doc_id NVARCHAR(50),
            invoice_attached VARCHAR(max),
            
            CONSTRAINT FK_invoice_attached_CO FOREIGN KEY (doc_id) REFERENCES CO(doc_id)
        );
    END
""")

connection.commit()  # Commit các lệnh tạo bảng

# Đóng kết nối
cursor.close()
connection.close()

print("Cơ sở dữ liệu và các bảng đã được tạo thành công.")


Cơ sở dữ liệu và các bảng đã được tạo thành công.


In [35]:
# Kết nối đến SQL Server
connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=COData;UID={username};PWD={password}')
cursor = connection.cursor()

# Chèn dữ liệu vào bảng CO
for idx,record in df.iterrows():
    cursor.execute("""
        MERGE INTO CO AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source
            (doc_id, Trạng_thái_hồ_sơ, Tax_code, Issuing_Authority, Form, Importing_Country, Reference_No, Issuance_date, Certified_date, total_value, total_quantity, total_weight, is_show_fob_value, declaration_place)
        ON target.doc_id = source.doc_id
        WHEN MATCHED THEN 
            UPDATE SET 
                Trạng_thái_hồ_sơ = source.Trạng_thái_hồ_sơ,
                Tax_code = source.Tax_code,
                Issuing_Authority = source.Issuing_Authority,
                Form = source.Form,
                Importing_Country = source.Importing_Country,
                Reference_No = source.Reference_No,
                Issuance_date = source.Issuance_date,
                Certified_date = source.Certified_date,
                total_value = source.total_value,
                total_quantity = source.total_quantity,
                total_weight = source.total_weight,
                is_show_fob_value = source.is_show_fob_value,
                declaration_place = source.declaration_place
        WHEN NOT MATCHED THEN 
            INSERT (doc_id, Trạng_thái_hồ_sơ, Tax_code, Issuing_Authority, Form, Importing_Country, Reference_No, Issuance_date, Certified_date, total_value, total_quantity, total_weight, is_show_fob_value, declaration_place)
            VALUES (source.doc_id, source.Trạng_thái_hồ_sơ, source.Tax_code, source.Issuing_Authority, source.Form, source.Importing_Country, source.Reference_No, source.Issuance_date, source.Certified_date, source.total_value, source.total_quantity, source.total_weight, source.is_show_fob_value, source.declaration_place);
    """, (
        record['doc_id'], str(record['Trạng thái hồ sơ']), record['Tax code'], 
        record['Issuing Authority'], record['Form'], record['Importing_Country'],
        record['Reference No.'], str(record['Issuance date']), str(record['Certified date']),
        record['total_value'], record['total_quantity'], record['total_weight'], 
        record['is_show_fob_value'], record['declaration_place']
    ))


    # Chèn dữ liệu vào bảng Transport
    cursor.execute("""
        INSERT INTO Transport (doc_id, transport_type, departure_date, vessel_aircraft_name, port_of_loading, port_of_discharge, transport_doc)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, record['doc_id'], record['transport_type'], str(record['Departure Date']), record['vessel_aircraft_name'], record['port_of_loading'], record['port_of_discharge'], record['transport_doc'])

    # Chèn dữ liệu vào bảng ExportDeclaration
    for export in record['ExportDeclaration']:
        export_invoice_date = convert_to_date(export.get('invoice_date', None))  # Kiểm tra ngày hóa đơn
        cursor.execute("""
            INSERT INTO ExportDeclaration (doc_id, item_num, invoice_number, invoice_date, invoice_link)
            VALUES (?, ?, ?, ?, ?)
        """, record['doc_id'], export['item_num'], export['invoice_number'], str(export_invoice_date), export['invoice_link'])

    # Chèn dữ liệu vào bảng Goods_info
    for idx, goods in enumerate(record['goods_info'][1:], 1):  # Bỏ qua dòng tiêu đề
        cursor.execute("""
            INSERT INTO Goods_info (doc_id, item_num, item_number, item_id, Marks_and_numbers_on_packages, Goods_description, Origin_criterion, FOB_value, Invoice_number_and_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, record['doc_id'], idx, goods[0], goods[1], goods[2], goods[3], goods[4], goods[5], goods[6])

    # Chèn dữ liệu vào bảng Remarks_info, nếu tồn tại
    remarks_info = record.get('remarks_info', [])  # Nếu không có 'remarks_info', gán giá trị mặc định là list rỗng
    for remark in remarks_info:
        cursor.execute("""
            INSERT INTO Remarks_info (doc_id, label, checked)
            VALUES (?, ?, ?)
        """, record['doc_id'], remark['label'], remark['checked'])

    # Chèn dữ liệu vào bảng from_to
    def split_address(address_list):
        name = address_list[0].strip() if address_list else ''
        address1 = address_list[1].strip() if len(address_list) > 1 else ''
        address2 = address_list[2].strip() if len(address_list) > 2 else ''
        return name, address1, address2

    # Tách dữ liệu và chèn vào bảngfrom_to
    name_export, address_export, address_export2 = split_address(record['Goods consigned from'])
    name_import, address_import, address_import2 = split_address(record['Goods consigned to'])

    # Chèn dữ liệu vào bảng from_to
    cursor.execute("""
        INSERT INTO from_to (doc_id, name_export, address_export, address_export2, name_import, address_import, address_import2)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, record['doc_id'], name_export, address_export, address_export2, name_import, address_import, address_import2)

    #chèn dữ liệu bảng origindoc
    for idx, docs in enumerate(record['origindoc'][0:]):  
        cursor.execute("""
            INSERT INTO origindoc (doc_id,origindoc )
            VALUES (?, ?)
        """, record['doc_id'], docs)
        
    #chèn dữ liệu bảng invoice_attached

    for idx, invoice_attached in enumerate(record['invoice_attached'][0:]):  
        cursor.execute("""
            INSERT INTO invoice_attached (doc_id,invoice_attached )
            VALUES (?, ?)
        """, record['doc_id'], invoice_attached)
    
# Commit các thay đổi vào cơ sở dữ liệu
connection.commit()

# Đóng kết nối
cursor.close()
connection.close()

print("Dữ liệu đã được chèn thành công.")


Dữ liệu đã được chèn thành công.
